<a href="https://colab.research.google.com/github/toinnn/Torax_Xray-Health-Care/blob/main/Multi_modal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os


num_cores = os.cpu_count()

# Obtém informações sobre a CPU
cpu_info = !lscpu

# Imprime o número de núcleos e informações sobre a CPU
print(f"Número de núcleos da CPU: {num_cores}")

Número de núcleos da CPU: 2


In [6]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install bitsandbytes


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://test.pypi.org/simple/


## Load Lllama2

In [9]:
from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset_name = 'AlexanderDoria/novel17_test' #french novels
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

ImportError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
text = "Écrire un texte dans un style baroque sur la glace et le feu ### Assistant: Si j'en luis éton"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# º ***My Model*** :

In [ ]:
from Transformer_Decoder import decoder , Trainer
import torch
import torch.nn as nn
# from Transformer_Decoder import decoder , Trainer
from torch.utils.data import DataLoader , Dataset
from torch.nn.utils.rnn import pad_sequence
import json
# from torchvision.io.image import decode_png, read_image
from torchvision.io import decode_png, read_image , ImageReadMode
import torchvision.transforms as T
import polars as pl
from PIL import Image
import numpy as np
import pickle


class my_model(nn.Module):
    def __init__(self ,llm ,  device : torch.device = torch.device("cpu") ) -> None:
        super(my_model , self  ).__init__()
        vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
        # vits14.eval()
        # print(vits14.eval())
        self.encoder = vits14.to(device)
        self.transposter = nn.Linear(768 , 100)

        self.llm = llm


        self.device  = device

        self.transform_image = T.Compose([T.Resize(224),  T.Normalize([0.5], [0.5])] )

    def setDevice(self , device : torch.device) :
        self.encoder = self.encoder.to( device )
        self.decoder.setDevice( device )
        return


    def forward_fit(self, image  , text_input , max_lengh = 100):
        # print(f"Passou do Encoder image : { image.shape}")
        img2 = []
        for img in image :
            img = img.view(1, img.shape[0] , img.shape[1])
            img = torch.cat( [img,img,img] , dim = 0 )
            # print(f"img.shape = {img.shape}")
            # print(f"trasnform : {self.transform_image( img  )[:3].unsqueeze(0).shape }")
            img2 += [self.transform_image( img )[:3].unsqueeze(0) ]
        # image = [ self.encoder(self.transform_image( img.view(1 , img.shape[0] , img.shape[1]))[:3].unsqueeze(0) ).view(1,1,-1)     for img in image]
        image = [ self.encoder(img).view(1 , 1 , -1) for img in img2 ]
        enc   = torch.cat(image , dim = 0 )

        image_input = self.transposter(enc)

        llm_input = torch.cat([image_input , text_input ] , dim = 1)
        return self.llm(llm_input) #self.decoder.forward_fit(enc , enc , max_lengh)

    def forward(self, image , text_input , max_lengh = 100):
        # image = self.transform_image(image)[:3].unsqueeze(0)
        # enc = self.encoder(image)

        img2 = []
        for img in image :
            img = img.view(1, img.shape[0] , img.shape[1])
            img = torch.cat( [img,img,img] , dim = 0 )
            # print(f"img.shape = {img.shape}")
            # print(f"trasnform : {self.transform_image( img  )[:3].unsqueeze(0).shape }")
            img2 += [self.transform_image( img )[:3].unsqueeze(0) ]
        # image = [ self.encoder(self.transform_image( img.view(1 , img.shape[0] , img.shape[1]))[:3].unsqueeze(0) ).view(1,1,-1)     for img in image]
        image = [ self.encoder(img).view(1 , 1 , -1) for img in img2 ]
        enc   = torch.cat(image , dim = 0 )

        image_input = self.transposter(enc)
        llm_input = torch.cat([image_input , text_input ] , dim = 1)

        return self.llm(llm_input)